In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [3]:
input_size = len(x_train[0])
hidden_layer_nodes = 4
output_size = len(set(y))

In [4]:
weights = [
    np.random.randn(hidden_layer_nodes, input_size),
    np.random.randn(output_size, hidden_layer_nodes)
]

biases = [np.zeros(hidden_layer_nodes), np.zeros(output_size)]

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [6]:
def der_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [7]:
def softmax(x):
    y = np.exp(x - x.max())
    return y / y.sum()

In [8]:
def cross_entropy_loss(y, yHat):
    return -np.sum(y * np.log(yHat))

In [9]:
def one_hot_encoder(x, output_num):
    result = np.zeros(output_num)
    result[x] = 1
    return result

In [10]:
def calc_activation(x, weights):
    activations = []
    activation = x
    for idx, w in enumerate(weights):
        z = np.dot(w, activation) + biases[idx]
        if idx < len(weights) - 1:
            activation = sigmoid(z)
        else:
            activation = softmax(z)
        activations.append(activation)
    return activations

In [11]:
def feed_forward_sample(x, y):
    activations_arr = calc_activation(x, weights)
    activation = activations_arr[-1]
    one_hot_y = one_hot_encoder(y, 3)
    loss = cross_entropy_loss(one_hot_y, activation)

    one_hot_prediction = np.zeros_like(activation)
    one_hot_prediction[np.argmax(activation)] = 1
    return loss, one_hot_prediction

In [12]:
def feed_forward_dataset(x, y):
    loss_arr = np.empty(x.shape[0])
    one_hot_predictions = np.empty((x.shape[0], 3))

    for i in range(x.shape[0]):
        loss_arr[i], one_hot_predictions[i] = feed_forward_sample(x[i], y[i])

    print("\nAverage loss=", np.round(np.average(loss_arr), decimals=2))

    y_one_hot = np.zeros((y.size, 3))
    y_one_hot[np.arange(y.size), y] = 1

    correct_predictions = np.sum(y_one_hot * one_hot_predictions)
    correct_pred_percent = format((correct_predictions / y.shape[0]) * 100, ".2f")
    print("Accuracy (% of correct predictions):", correct_predictions, "/", y.shape[0], "[", correct_pred_percent, "%]")

In [13]:
def train_one_sample(x, y, lr_rate=0.001):
    activations = calc_activation(x, weights)
    a = activations[-1]
    one_hot_y = one_hot_encoder(y, 3)

    one_hot_guess = np.zeros_like(a)
    one_hot_guess[np.argmax(a)] = 1

    weight_gradients = [None] * len(weights)
    bias_gradients = [None] * len(weights)
    activation_gradients = [None] * (len(weights) - 1)

    for i in reversed(range(len(weights))):

        if i == len(weights) - 1:
            y = one_hot_y.reshape(-1, 1)
            a = activations[i].reshape(-1, 1)
            a_prev = activations[i - 1].reshape(-1, 1)

            weight_gradients[i] = np.dot((a - y), a_prev.T)
            bias_gradients[i] = a - y
        else:
            w_next = weights[i + 1]
            a_next = activations[i + 1].reshape(-1, 1)
            y = one_hot_y.reshape(-1, 1)
            a = activations[i].reshape(-1, 1)
            if i > 0:
                a_prev = activations[i - 1].reshape(-1, 1)
            else:
                a_prev = x.flatten().reshape(-1, 1)

            if i == len(weights) - 2:
                dA = np.matmul(w_next.T, (a_next - y))
                activation_gradients[i] = dA
            else:
                dA_next = activation_gradients[i + 1]
                dA = np.matmul(w_next.T, (der_sigmoid(a_next) * dA_next))
                activation_gradients[i] = dA

            z = der_sigmoid(a) * dA
            weight_gradients[i] = np.dot(z, a_prev.T)
            bias_gradients[i] = z

        weights[i] -= weight_gradients[i] * lr_rate
        biases[i] -= bias_gradients[i].flatten() * lr_rate

In [14]:
def train_dataset(learning_rate=0.0001):
    for idx in range(x_train.shape[0]):
        train_one_sample(x_train[idx], y_train[idx], learning_rate)

In [15]:
def train_test_epoch():
    train_dataset()
    feed_forward_dataset(x_test, y_test)

In [16]:
for i in range(1000):
    train_test_epoch()


Average loss= 0.9
Accuracy (% of correct predictions): 22.0 / 30 [ 73.33 %]

Average loss= 0.89
Accuracy (% of correct predictions): 22.0 / 30 [ 73.33 %]

Average loss= 0.88
Accuracy (% of correct predictions): 23.0 / 30 [ 76.67 %]

Average loss= 0.88
Accuracy (% of correct predictions): 23.0 / 30 [ 76.67 %]

Average loss= 0.87
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.87
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.86
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.86
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.85
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.85
Accuracy (% of correct predictions): 26.0 / 30 [ 86.67 %]

Average loss= 0.84
Accuracy (% of correct predictions): 27.0 / 30 [ 90.00 %]

Average loss= 0.84
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.84
Accuracy (% of correct predictions): 28.0 / 3


Average loss= 0.78
Accuracy (% of correct predictions): 20.0 / 30 [ 66.67 %]

Average loss= 0.78
Accuracy (% of correct predictions): 20.0 / 30 [ 66.67 %]

Average loss= 0.78
Accuracy (% of correct predictions): 20.0 / 30 [ 66.67 %]

Average loss= 0.77
Accuracy (% of correct predictions): 20.0 / 30 [ 66.67 %]

Average loss= 0.77
Accuracy (% of correct predictions): 20.0 / 30 [ 66.67 %]

Average loss= 0.77
Accuracy (% of correct predictions): 20.0 / 30 [ 66.67 %]

Average loss= 0.77
Accuracy (% of correct predictions): 20.0 / 30 [ 66.67 %]

Average loss= 0.77
Accuracy (% of correct predictions): 21.0 / 30 [ 70.00 %]

Average loss= 0.77
Accuracy (% of correct predictions): 21.0 / 30 [ 70.00 %]

Average loss= 0.77
Accuracy (% of correct predictions): 21.0 / 30 [ 70.00 %]

Average loss= 0.77
Accuracy (% of correct predictions): 21.0 / 30 [ 70.00 %]

Average loss= 0.77
Accuracy (% of correct predictions): 21.0 / 30 [ 70.00 %]

Average loss= 0.77
Accuracy (% of correct predictions): 21.0 / 


Average loss= 0.72
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.72
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.72
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.72
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.72
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.72
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.72
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.72
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.72
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.72
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.71
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.71
Accuracy (% of correct predictions): 24.0 / 30 [ 80.00 %]

Average loss= 0.71
Accuracy (% of correct predictions): 24.0 / 


Average loss= 0.67
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.67
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.67
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.67
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.67
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.67
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.67
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.67
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.67
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.67
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.67
Accuracy (% of correct predictions): 25.0 / 30 [ 83.33 %]

Average loss= 0.67
Accuracy (% of correct predictions): 26.0 / 30 [ 86.67 %]

Average loss= 0.67
Accuracy (% of correct predictions): 26.0 / 


Average loss= 0.64
Accuracy (% of correct predictions): 27.0 / 30 [ 90.00 %]

Average loss= 0.64
Accuracy (% of correct predictions): 27.0 / 30 [ 90.00 %]

Average loss= 0.64
Accuracy (% of correct predictions): 27.0 / 30 [ 90.00 %]

Average loss= 0.64
Accuracy (% of correct predictions): 27.0 / 30 [ 90.00 %]

Average loss= 0.64
Accuracy (% of correct predictions): 27.0 / 30 [ 90.00 %]

Average loss= 0.64
Accuracy (% of correct predictions): 27.0 / 30 [ 90.00 %]

Average loss= 0.63
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.63
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.63
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.63
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.63
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.63
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.63
Accuracy (% of correct predictions): 28.0 / 

Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.61
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.61
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.61
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.61
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.61
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.61
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.61
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.61
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.61
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.61
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.6
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.6
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average


Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.58
Accuracy (% of correct predictions): 28.0 / 


Average loss= 0.56
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.56
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.56
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.56
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.56
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.56
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.56
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.56
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.56
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.56
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.56
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.56
Accuracy (% of correct predictions): 28.0 / 30 [ 93.33 %]

Average loss= 0.56
Accuracy (% of correct predictions): 28.0 / 


Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 30 [ 96.67 %]

Average loss= 0.54
Accuracy (% of correct predictions): 29.0 / 